## Building deep neural network with more than one hidden layer

In [23]:
import h5py
import matplotlib.pyplot as plt


In [22]:
! pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2979 sha256=8cc38fbdede94c35f97b66dca4add521c24b9770550f512ba632b932ca68274b
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\06\7e\ad\1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [24]:
import import_ipynb
import lab4testcases_part1

importing Jupyter notebook from lab4testcases_part1.ipynb


In [2]:
import numpy as np

def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    
    return A, Z

def relu(Z):

    A = np.maximum(0,Z)

    return A, Z


def relu_backward(dA, Z):

    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoid_backward(dA, Z):

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    return dZ

### Initialization

**2-layer Network**

In [3]:
def initialize_parameters(n_x,n_h,n_y):
    np.random.seed(1)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros(n_h,1)
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros(n_y,1)
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

**L-layer Network**

In [4]:
def initialize_params_deep(layer_dims):
    np.random.seed(3)
    parameters={}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters["W"+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters["b"+str(l)] = np.zeros(layer_dims[l])
    return parameters

In [5]:
parameters = initialize_params_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [0. 0. 0. 0.]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [0. 0. 0.]


### Forward propogation Module

In [47]:
def linear_forward(A,W,b):
    print(W.shape,A.shape)
    Z = np.dot(W,A)+b
    cache = (A,W,b)
    return Z, cache

In [48]:
A, W, b = lab4testcases_part1.linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

(1, 3) (3, 2)
Z = [[ 3.26295337 -1.23429987]]


### Linear Activation Forward

In [59]:
def linear_activation_forward(A_prev,W,b,activation):
    if activation == 'relu':
        print("linear activation forward")
        print(A_prev.shape)
        print(W.shape)
        Z,linearcache = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
    elif activation == 'sigmoid':
        Z,linearcache = linear_forward(A_prev,W,b)
        A,activation_cache =sigmoid(Z)
    cache = (linearcache,activation_cache)
    return A,cache

In [55]:
A_prev, W, b = lab4testcases_part1.linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

(1, 3) (3, 2)
With sigmoid: A = [[0.96890023 0.11013289]]
linear activation forward
(3, 2)
(1, 3) (3, 2)
With ReLU: A = [[3.43896131 0.        ]]


### L-Layer Model

In [70]:
def l_model_forward(X,parameters):
    caches=[]
    A = X
    l = len(parameters)//2 # because we have both w and b in params, so divided by 2
    for i in range(1,l):
        A_prev = A
        print("In the model forward")
        print(A_prev.shape)
        W = parameters['W' + str(i)]
        b = parameters['b' + str(i)]
        print(W.shape)
        A,cache = linear_activation_forward(A_prev,W,b,activation='relu')
        caches.append(cache)
    W = parameters['W' + str(l)]
    b = parameters['b' + str(l)]
    A_final,cache = linear_activation_forward(A,W,b,activation='sigmoid')
    caches.append(cache)
    return A_final,caches
    

In [71]:
X, parameters = lab4testcases_part1.L_model_forward_test_case_2hidden()
#print(X.shape)
#print(parameters)
AL, caches = l_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

In the model forward
(5, 4)
(4, 5)
linear activation forward
(5, 4)
(4, 5)
(4, 5) (5, 4)
In the model forward
(4, 4)
(3, 4)
linear activation forward
(4, 4)
(3, 4)
(3, 4) (4, 4)
(1, 3) (3, 4)
AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


### Cost Function

In [72]:
def compute_cost(AL, Y):
    print("AL_shape = "+str(AL.shape))   #(1,3)
    print("Y_shape = "+str(Y.shape))     #(1,3)
    m=Y.shape[1]
    cost = (-1/m) * (np.dot(Y,np.log(AL).T) + np.dot(1-Y,np.log(1-AL).T))
    cost = np.squeeze(cost)
    return cost

In [74]:
Y, AL = lab4testcases_part1.compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

AL_shape = (1, 3)
Y_shape = (1, 3)
cost = 0.41493159961539694


### Backward Propogation Module

In [78]:
def linear_backward(dZ, cache):
    A_prev, W,b = cache
    m = A_prev.shape[1]
    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1)
    dA_prev = np.dot(W.T,dZ)
    return dA_prev,dW,db

In [79]:
# Set up some test inputs
dZ, linear_cache = lab4testcases_part1.linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [0.50629448]


In [80]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":

        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

        
    elif activation == "sigmoid":
  
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [81]:
dAL, linear_activation_cache = lab4testcases_part1.linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [-0.05729622]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [-0.20837892]


In [86]:
def l_model_backward(AL,Y,caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    print("L"+str(L))
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    #last layer sigmoid
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    print("dA"+ str(L-1)+" = "+str(grads["dA" + str(L-1)]))
    print("dW"+ str(L)+" = "+str(grads["dW" + str(L)]))
    print("db"+ str(L)+" = "+str(grads["db" + str(L)]))
    
    # rest of the layers
    for i in reversed(range(L-1)):
        current_cache = caches[i]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(i + 1)], current_cache, activation = "relu")
        grads["dA" + str(i)] = dA_prev_temp
        grads["dW" + str(i + 1)] = dW_temp
        grads["db" + str(i + 1)] = db_temp
        
    return grads

In [88]:
AL, Y_assess, caches = lab4testcases_part1.L_model_backward_test_case()
grads = l_model_backward(AL, Y_assess, caches)
grads

L2
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]
dW2 = [[-0.39202432 -0.13325855 -0.04601089]]
db2 = [0.15187861]


{'dA1': array([[ 0.12913162, -0.44014127],
        [-0.14175655,  0.48317296],
        [ 0.01663708, -0.05670698]]),
 'dW2': array([[-0.39202432, -0.13325855, -0.04601089]]),
 'db2': array([0.15187861]),
 'dA0': array([[ 0.        ,  0.52257901],
        [ 0.        , -0.3269206 ],
        [ 0.        , -0.32070404],
        [ 0.        , -0.74079187]]),
 'dW1': array([[0.41010002, 0.07807203, 0.13798444, 0.10502167],
        [0.        , 0.        , 0.        , 0.        ],
        [0.05283652, 0.01005865, 0.01777766, 0.0135308 ]]),
 'db1': array([-0.22007063,  0.        , -0.02835349])}

### Update Parameters

In [94]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters)//2
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [95]:
parameters, grads = lab4testcases_part1.update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
